# Crest factor analysis
### 17/02/2020
### Laurent @IRAP
This Notebook checks the impact of a frequency not aligned with Fs/2**N:

In [93]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft

%matplotlib widget

# Definition of some constants
Fs=19530000 # sampling frequency
Fmin=1e6 # minimum carrier frequency
Fmax=5e6 # maximum carrier frequency
Ncar=40 # Number of carriers

# Definition of colors
c=['r', 'orange', 'gold', 'darkorchid', 'mediumpurple', 'steelblue', 'b', 'turquoise', 'g', 'gray', 'dark']


In [94]:
# Computation of crest factor
def crest_factor(signal):
    peak = abs(signal).max()
    return(peak / signal.std())


In [116]:
# Definition of carrier frequencies
def gen_freq(Fmin, Fmax, Fres, Fsigma):
    Fstep=(Fmax-Fmin)/Ncar
    F=Fmin+Fstep*np.arange(Ncar)+np.random.normal(loc=0, scale=Fsigma, size=Ncar)
    F=Fres*np.floor(F/Fres) # quantization
    return(F)

# Computation of optimised bias signal
def make_bias(Npts, Fres, Niter, Fsigma=2e3):
    t=np.arange(Npts)/Fs
    freq=gen_freq(Fmin, Fmax, Fres, Fsigma)
    Cf_tot=0
    Cf_opt=np.inf
    for i in range(Niter):
        bias=np.zeros(Npts)
        phase=np.random.rand(Ncar)*2.*np.pi
        for i in range(Ncar):
            bias+=np.sin(2.*np.pi*freq[i]*t+phase[i])
        Cf=crest_factor(bias)
        Cf_tot+=Cf
        if Cf<Cf_opt:
            Cf_opt=Cf
            phase_opt=phase
    return(Cf_opt, Cf_tot/Niter, bias, freq, phase_opt)

# Spectral analisys of a bias signal
def spectral(bias, title):
    bias_f=abs(fft(bias))
    #plt.plot(bias_f)
    not_zero=np.where(bias_f != 0)[0]

    bias_f_dB=-300*np.ones(len(bias_f))
    bias_f_dB[not_zero]=20.*np.log10(bias_f[not_zero])
    bias_f_dB-=bias_f_dB.max()
    fig = plt.figure(figsize=(9, 7)) 
    ax = plt.subplot(1,1,1)
    ax.plot(bias_f_dB)
    ax.set_title(title)
    ax.set_ylabel("")
    ax.set_ylim(-150,10)
    ax.grid(color='k', linestyle=':', linewidth=0.5)


In [117]:
print(Fs/2**14, Fs/2**20)


1192.0166015625 18.625259399414062


In [118]:
Npts=2**20
Fres=(Fs/2**14)
Cf_opt, _, bias, _, _=make_bias(Npts, Fres, Niter=10, Fsigma=2e3)
bias=np.round(2**16*(bias/(bias.max()-bias.min())))
spectral(bias, 'Fres=Fs/(2**14)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [138]:
Npts=2**20
Fres=(Fs/2**20+Fs/2**14)
Cf_opt, _, bias, ftab, phitab=make_bias(Npts, Fres, Niter=10, Fsigma=2e3)
bias=np.round(2**16*(bias/(bias.max()-bias.min())))
#spectral(bias, 'Fres=Fs/(2**20+2**14)')


print("Cf = ", Cf_opt)

# Printing frequencies
print("freqtab=[", end='')
nlignes=5
ncol=8
for ligne in range(nlignes-1):
    print("    ")
    for col in range(ncol):
        print("{0:8.5f}, ".format(ftab[col+ligne*ncol]), end = '')
print("    ")
for col in range(ncol-1):
    print("{0:8.5f}, ".format(ftab[col+(nlignes-1)*ncol]), end = '')
print("{0:8.5f} ".format(ftab[ncol*nlignes-1]))
print("];")

# Printing phases (deg)
phitab=phitab*180/np.pi
print("phitab=[", end='')
nlignes=5
ncol=8
for ligne in range(nlignes-1):
    print("    ")
    for col in range(ncol):
        print("{0:8.5f}, ".format(phitab[col+ligne*ncol]), end = '')
print("    ")
for col in range(ncol-1):
    print("{0:8.5f}, ".format(phitab[col+(nlignes-1)*ncol]), end = '')
print("{0:8.5f} ".format(phitab[ncol*nlignes-1]))
print("];")


Cf =  3.3326942790726974
freqtab=[    
1002411.46088, 1101684.09348, 1200956.72607, 1297808.07495, 1399501.99127, 1498774.62387, 1602889.82391, 1697319.88907,     
1800224.44725, 1897075.79613, 2002401.63803, 2099252.98691, 2199736.26137, 2301430.17769, 2397070.88470, 2496343.51730,     
2596826.79176, 2702152.63367, 2799003.98254, 2894644.68956, 2996338.60588, 3096821.88034, 3197305.15480, 3301420.35484,     
3400692.98744, 3501176.26190, 3598027.61078, 3698510.88524, 3792940.95039, 3899477.43416, 3998750.06676, 4096812.05750,     
4199716.61568, 4301410.53200, 4400683.16460, 4499955.79720, 4601649.71352, 4702132.98798, 4798984.33685, 4897046.32759 
];
phitab=[    
277.23498,  7.30595, 184.02708, 116.07985, 271.29163, 201.37568, 80.00372, 330.46316,     
258.41903, 101.07190, 16.50202, 60.37930, 177.31398, 50.29364, 349.66483, 150.34686,     
240.29073, 111.05765, 30.82543, 16.23958, 295.46884, 276.54160, 147.28161, 231.50081,     
131.77617, 318.90882, 296.02194, 83.09532, 157.06794,

In [137]:
print(Fres)
print(ftab-Fres*(ftab/Fres))

1210.641860961914
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
